1. NoSQL Databases:
   a. Write a Python program that connects to a MongoDB database and inserts a new document into a collection named "students". The document should include fields such as "name", "age", and "grade". Print a success message after the insertion.
   b. Implement a Python function that connects to a Cassandra database and inserts a new record into a table named "products". The record should contain fields like "id", "name", and "price". Handle any potential errors that may occur during the insertion.


In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import SimpleStatement

def insert_product_into_cassandra(product):
    # Connect to Cassandra
    cloud_config = {
        'secure_connect_bundle': 'path_to_secure_bundle.zip',  # Replace with your secure connect bundle path
        'username': 'your_username',  # Replace with your Cassandra username
        'password': 'your_password'  # Replace with your Cassandra password
    }

    auth_provider = PlainTextAuthProvider(
        username=cloud_config['username'],
        password=cloud_config['password']
    )
    cluster = Cluster(cloud=cloud_config['secure_connect_bundle'], auth_provider=auth_provider)
    session = cluster.connect()

    # Prepare the INSERT statement
    insert_statement = SimpleStatement(
        "INSERT INTO products (id, name, price) VALUES (?, ?, ?)",
        consistency_level=1  # Set the desired consistency level
    )

    # Execute the INSERT statement
    try:
        session.execute(insert_statement, (product['id'], product['name'], product['price']))
        print("Record inserted successfully.")
    except Exception as e:
        print("Failed to insert record:", e)

    # Close the Cassandra connection
    session.shutdown()
    cluster.shutdown()

# Example usage
product = {
    'id': 'P001',
    'name': 'Product Name',
    'price': 9.99
}

insert_product_into_cassandra(product)



2. Document-oriented NoSQL Databases:
   a. Given a MongoDB collection named "books", write a Python function that fetches all the books published in the last year and prints their titles and authors.
   b. Design a schema for a document-oriented NoSQL database to store customer information for an e-commerce platform. Write a Python program to insert a new customer document into the database and handle any necessary validations.


In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta

def fetch_recent_books():
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Replace with your MongoDB connection details

    # Get the "books" collection
    db = client['your_database_name']  # Replace with your MongoDB database name
    collection = db['books']

    # Calculate the date range for the last year
    current_date = datetime.now()
    last_year = current_date - timedelta(days=365)

    # Fetch books published in the last year
    recent_books = collection.find({'publish_date': {'$gt': last_year}})

    # Print the titles and authors of the recent books
    for book in recent_books:
        print("Title:", book['title'])
        print("Author:", book['author'])
        print()

    # Close the MongoDB connection
    client.close()

# Call the function to fetch recent books
fetch_recent_books()



3. High Availability and Fault Tolerance:
   a. Explain the concept of replica sets in MongoDB. Write a Python program that connects to a MongoDB replica set and retrieves the status of the primary and secondary nodes.
   b. Describe how Cassandra ensures high availability and fault tolerance in a distributed database system. Write a Python program that connects to a Cassandra cluster and fetches the status of the nodes.


In [ ]:
from pymongo import MongoClient

# Connect to MongoDB Replica Set
client = MongoClient('mongodb://localhost:27017,localhost:27018,localhost:27019/?replicaSet=your_replica_set_name')

# Get the status of the replica set
repl_status = client.admin.command('replSetGetStatus')

# Print the status of each node
for member in repl_status['members']:
    if member['stateStr'] == 'PRIMARY':
        print(f"Primary Node: {member['name']}")
    elif member['stateStr'] == 'SECONDARY':
        print(f"Secondary Node: {member['name']}")

# Close the MongoDB connection
client.close()


from cassandra.cluster import Cluster

# Connect to Cassandra Cluster
cluster = Cluster(['localhost'])  # Replace with your Cassandra cluster contact points

# Retrieve the metadata and get the cluster's current ring information
metadata = cluster.metadata
ring = metadata.token_map.ring

# Print the status of each node
for token, (host, _) in ring.items():
    print(f"Node: {host}")

# Close the Cassandra connection
cluster.shutdown()




4. Sharding in MongoDB:
   a. Explain the concept of sharding in MongoDB and how it improves performance and scalability. Write a Python program that sets up sharding for a MongoDB cluster and inserts multiple documents into a sharded collection.
   b. Design a sharding strategy for a social media application where user data needs to be distributed across multiple shards. Write a Python program to demonstrate how data is distributed and retrieved from the sharded cluster.


a. Sharding in MongoDB:
Sharding in MongoDB is a technique used to horizontally partition data across multiple servers or shards. Each shard contains a subset of the data, allowing MongoDB to scale horizontally by distributing the data and workload across multiple machines.

Sharding improves performance and scalability in MongoDB in the following ways:

Increased Data Volume: Sharding allows MongoDB to handle large amounts of data that exceed the capacity of a single server. By distributing the data across multiple shards, MongoDB can store and process larger datasets.

Improved Performance: Sharding distributes the data and workload across multiple shards, enabling parallel processing and reducing the load on individual shards. This improves read and write performance by allowing queries to be executed in parallel across multiple shards.

Scalability: Sharding provides horizontal scalability, allowing you to add more shards as your data grows or as the workload increases. By adding more shards, you can distribute the data and workload evenly, maintaining high performance and accommodating additional data and traffic.

In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Replace with your MongoDB connection details

# Connect to the sharded cluster
db = client['your_database_name']  # Replace with your database name

# Retrieve user data based on the sharding key
user_id = 'user_id_to_retrieve'  # Replace with the user ID to retrieve

# Find the shard key range for the user ID
shard_key_range = db['your_shard_key_range_collection'].find_one({'_id': user_id})

# Determine the shard containing the user data
shard_name = shard_key_range['shard']

# Connect to the shard containing the user data
shard_client = client[shard_name]

# Retrieve the user data from the shard
user_data = shard_client['your_user_collection'].find_one({'_id': user_id})

# Print the retrieved user data
print(user_data)

# Close the MongoDB connection
client.close()



5. Indexing in MongoDB:
   a. Describe the concept of indexing in MongoDB and its importance in query optimization. Write a Python program that creates an index on a specific field in a MongoDB collection and executes a query using that index.
   b. Given a MongoDB collection named "products", write a Python function that searches for products with a specific keyword in the name or description. Optimize the query by adding appropriate indexes.


a. Indexing in MongoDB:
Indexing in MongoDB is the process of creating additional data structures to improve the efficiency of data retrieval operations. Indexes are created on specific fields in a collection, allowing MongoDB to locate and access data more quickly.

Indexes play a crucial role in query optimization by reducing the number of documents that need to be examined when executing a query. They help MongoDB identify the subset of documents that satisfy the query conditions, improving the overall query performance.

Creating an index involves analyzing the values in the indexed field and building a data structure that maps those values to their corresponding document locations. This data structure allows MongoDB to perform index-based queries efficiently.

Here's a Python program that creates an index on a specific field in a MongoDB collection and executes a query using that index:

In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Replace with your MongoDB connection details

# Get the collection
db = client['your_database_name']  # Replace with your database name
collection = db['your_collection_name']  # Replace with your collection name

# Create an index on the desired field
index_field = 'name'  # Replace with the field to create an index on
index_name = f"{index_field}_index"
collection.create_index([(index_field, 1)], name=index_name)

# Execute a query using the index
query = {index_field: 'search_value'}  # Replace with the search value
result = collection.find(query)

# Print the query results
for document in result:
    print(document)

# Close the MongoDB connection
client.close()
